<a href="https://colab.research.google.com/github/jhodgens17/audio_classification/blob/main/Second_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
#Import packages
import os.path
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
#Getting a list of files containing audio clips for 'up'
up_files = os.listdir('/content/drive/MyDrive/train/audio/up')

In [4]:
#Getting the full path for the 'up' speech clips
up_paths = []
for file in up_files:
  path = '/content/drive/MyDrive/train/audio/up/' + file
  up_paths.append(path)

In [5]:
#Repeating the process for 'down' clips
down_files = os.listdir('/content/drive/MyDrive/train/audio/down')
down_paths = []
for file in down_files:
  path = '/content/drive/MyDrive/train/audio/down/' + file
  down_paths.append(path)

In [9]:
#Defining function to repeat process to get waveform for each clip
def get_waveform(path):
  audio_binary = tf.io.read_file(path)
  audio, sample_rate = tf.audio.decode_wav(audio_binary)
  waveform = tf.squeeze(audio, axis=-1)
  return waveform

In [6]:
#Defining a function that makes all audio clips same length to get spectrogram
def get_spectrogram(waveform):
  zero_padding = tf.zeros([16000] - tf.shape(waveform), dtype=tf.float32)
  equal_length = tf.concat([waveform, zero_padding], 0)
  spectrogram = tf.signal.stft(equal_length, frame_length=255, frame_step=128)
  spectrogram = tf.abs(spectrogram)
  return spectrogram

In [7]:
#Getting all of the files
up_files_ds = tf.data.Dataset.from_tensor_slices(up_paths)
down_files_ds = tf.data.Dataset.from_tensor_slices(down_paths)

In [10]:
#Getting the waveform for each file
up_waveform_ds = up_files_ds.map(get_waveform)
down_waveform_ds = down_files_ds.map(get_waveform)

In [11]:
#Getting the spectrogram for each file
up_spectrogram_ds = up_waveform_ds.map(get_spectrogram)
down_spectrogram_ds = down_waveform_ds.map(get_spectrogram)

In [18]:
X = []
y = []
for element in up_spectrogram_ds:
  X.append(element.numpy())
  y.append(1)

In [21]:
for element in down_spectrogram_ds:
  X.append(element.numpy())
  y.append(0)

In [24]:
X = np.array(X)
y = np.array(y)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [109]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers.experimental import preprocessing

In [110]:
model = Sequential()

In [95]:
for spectrogram in up_spectrogram_ds.take(1):
  input_shape = spectrogram.shape
print('Input shape:', input_shape)

Input shape: (124, 129)


In [111]:
model.add(Input(shape=input_shape))
model.add(Flatten())
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [112]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [113]:
model.fit(X_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
56/56 [==============================] - 1s 5ms/step - loss: 0.5898 - accuracy: 0.6941
Epoch 2/10
56/56 [==============================] - 0s 5ms/step - loss: 0.3778 - accuracy: 0.8893
Epoch 3/10
56/56 [==============================] - 0s 5ms/step - loss: 0.2753 - accuracy: 0.9200
Epoch 4/10
56/56 [==============================] - 0s 5ms/step - loss: 0.2029 - accuracy: 0.9420
Epoch 5/10
56/56 [==============================] - 0s 5ms/step - loss: 0.1525 - accuracy: 0.9603
Epoch 6/10
56/56 [==============================] - 0s 5ms/step - loss: 0.1129 - accuracy: 0.9682
Epoch 7/10
56/56 [==============================] - 0s 5ms/step - loss: 0.0928 - accuracy: 0.9772
Epoch 8/10
56/56 [==============================] - 0s 5ms/step - loss: 0.0830 - accuracy: 0.9786
Epoch 9/10
56/56 [==============================] - 0s 5ms/step - loss: 0.0641 - accuracy: 0.9828
Epoch 10/10
56/56 [==============================] - 0s 5ms/step - loss: 0.0569 - accuracy: 0.9865


In [119]:
y_pred = np.round(model.predict(X_test))

In [120]:
accuracy_score(y_test, y_pred)

0.9527027027027027